# Intro to the ML_Pipeline

Here we will walk through how to use scripts in the ML_Pipeline to do machine learning.
We are interested in trying to predict grain yield in Zea maize (corn) using information about
gene expression levels for 31,237 genes measured on seedlings. We will use these data to 
predict:
1. Grain yield for each line (i.e. a regression problem)
2. If the line was in the top 25% for yield (i.e. a binary classification problem)
3. If the line was in the top 25%, bottom 25%, or middle 50% for yield (i.e. a multi-class classification problem).


In [5]:
%run holdout.py -df Tutorial/example_df_binary.txt -type c -p 0.1

Holding out 10.0 percent
Pulling holdout set from classes: [1 0]
['AT2G25140', 'ATCG00180', 'AT1G64320', 'AT3G61220', 'AT2G47670', 'AT4G14590', 'AT5G07640', 'AT3G11600', 'AT3G04350', 'AT1G48750', 'AT2G16580', 'AT1G62510', 'AT4G18350', 'AT1G80320', 'AT1G16705', 'AT1G17744', 'AT3G19270', 'AT2G02350', 'AT1G56600', 'AT3G14200', 'AT5G06550', 'AT4G27850', 'AT4G13195', 'AT1G67120', 'AT5G58470', 'AT4G01450', 'AT5G44572', 'AT1G04570']
['AT2G25140', 'ATCG00180', 'AT1G64320', 'AT3G61220', 'AT2G47670', 'AT4G14590', 'AT5G07640', 'AT3G11600', 'AT3G04350', 'AT1G48750', 'AT2G16580', 'AT1G62510', 'AT4G18350', 'AT1G80320', 'AT1G16705', 'AT1G17744', 'AT3G19270', 'AT2G02350', 'AT1G56600', 'AT3G14200', 'AT5G06550', 'AT4G27850', 'AT4G13195', 'AT1G67120', 'AT5G58470', 'AT4G01450', 'AT5G44572', 'AT1G04570', 'AT5G57360', 'AT4G28652', 'AT4G15990', 'AT2G43700', 'AT2G04622', 'AT5G41420', 'AT2G28290', 'AT1G08060', 'AT2G01560', 'AT5G64010', 'AT5G53930', 'AT1G79350', 'AT4G22860', 'AT2G18969', 'AT5G52930', 'AT2G43260